# Backend Analysis Einfluss städtischer Faktoren auf Hitzeentwicklung (Urban Heat)

## Ziel

Analyse und modellierung, welche Faktoren (Versiegelung, geparkte Fahrzeuge, Dachtypen, Vegetation, Morphologie) die lokale Oberflächen- und Lufttemperatur in einer Stadt beeinflussen. Reproduzierbare Ergebnisse produzieren, interaktive Visualisierungen und eine kurze Policy-/Design-Empfehlung für Klimaanpassung und Flächengerechtigkeit 

## Wichtige Einflussfaktoren (aus Literatur)

- Versiegelung / Bodenbelag (Asphalt, Beton, permeable Flächen)
- Geparkte Fahrzeuge (Dichte von Parkflächen / On-street parking)
- Dachtypen & -materialien (Flach vs. Steil, Metall/Bitumen vs. Grün-/Kühl-Dächer)
- Vegetationsbedeckung / Baumkronendichte (NDVI, Baumhöhen)
- Gebäudehöhe & Dichte (Sky View Factor, canyon geometry)
- Bodenfeuchte / Grundwasser (Einfluss auf Evapotranspiration)
- Oberflächentemperatur (Land Surface Temperature, LST)
- Straßenbreite & Material, Albedo von Oberflächen
- Anthropogene Wärmequellen (Verkehrsdichte, Industrie, Klimaanlagenabwärme)
- Mikroklima-Elemente: Schatten (Baum-/Gebäudeschatten), Windkorridore
- Sozioökonomische Faktoren (Vulnerabilität, Alter, Einkommen) für Flächengerechtigkeit

## Datenquellen (open/praktikabel ??)

- Satellit: Landsat (30m + Thermal, LST), Sentinel-2 (10–20m multispectral), MODIS (Tages-Trends)
- Google Earth Engine (GEE) — großer Vorteil für Vorverarbeitung/Time-series
- OpenStreetMap — Straßen, Parkplätze, Landuse
- Building footprints & heights: Microsoft Building Footprints, lokale Kataster, LiDAR (falls verfügbar)
- Impervious surface maps (Copernicus, Global Surface Water / Imperviousness)
- Aerial imagery / hochaufgelöste Orthophotos (kommunal, falls verfügbar) für Parkplätze & Dächer
- Verkehr & Parken: Verkehrszählungen, Parkraumbewirtschaftung, Mobilitätsdaten (falls vorhanden)
- Meteorologie: lokale Wetterstationen, ERA5 (reanalysen)
- Socio-demographic: Census / Mikrodaten (für Equity-Analysen)
- Street-level photos / Google Street View (Dachmaterial, Parkmodus) — nur falls Lizenz erlaubt


## To Do
- LST aus Landsat / GEE extrahieren (Cloud-Masking, atmosph. Korrektur).
- Vegetationsindizes (NDVI, EVI).
- Imperviousness / Landuse-Klassifikation / Parkflächen-Mapping (object detection oder semantic segmentation auf Luftbildern).
- Gebäude-Features: footprint area, height (falls vorhanden), roof-area ratio.
- Straßen- & Parkraum-Dichte (Buffers / Kernel density).
- Meteorologische Normierung (z. B. LST an Referenzstation anpassen)
- Raster- oder Tabellen-Features pro Analyse-Unit (z. B. 30m-Pixel, 100m Grid, oder Gebäude-/Block-Ebene)
    
**Explorative Analyse:** Korrelationen, spatial plots, Hotspot-Analyse, Moran’s I für räumliche Autokorrelation.

**Modellierung:**
- Baseline: Lineare Regression / OLS (mit räumlicher Fehleranalyse
- Tree-based: Random Forest, Gradient Boosting (LightGBM/XGBoost) für Feature Importance
- Räumlich explizite Modelle: Spatial Lag / Spatial Error, Geographically Weighted Regression (GWR)
- Zeitserie/Peak-Analyse: LSTM/Temporal models falls viele Zeitpunkte.
- Causal-Inferenz-Ansätze (wenn Interventionen vorhanden): Difference-in-Differences, Matching
    
**Validierung:**
- Spatial cross-validation (nicht random), z. B. block-CV
- Metriken: RMSE, MAE, R², sowie Fehleranalyse nach Quartilen/Sozioökonomischen Gruppen

        
**Interpretierbarkeit:** SHAP, PDPs, lokale Erklärungen (LIME) und Karten der lokalen Effekte.
    
**Szenario-/Interventionsmodellierung:**
Simuliere Effekte: Begrünung, Kühl-/weiße Dächer, Reduktion Parkflächen, erhöhte Albedo. Quantifizierung von Temperaturänderungen regional und vs. vulnerable Gruppen
    
**Visualisierung:**
- Interaktive Karte (Kepler.gl / folium / deck.gl), Dashboard (Streamlit / Panel).
- Policy-Recommendations + Equity-Fokus

## Feature Engineering:
Pufferstatistiken (%Versiegelung innerhalb 100m, Baumkronendichte 50m).
Morphologische Indices: SVF, canyon height-to-width ratio.
Zeitliche Features: Tageszeit, Sonneneinstrahlung, Hitzeindex.

    target: LST_pixel oder Lufttemperatur (Messnetz??)
    % Imperviousness (100m buffer)
    NDVI_mean (50m)
    Tree_canopy_fraction
    Building_density (m²/m²)
    Mean_building_height
    Road_density
    On-street_parking_density (vector kernel)
    Vehicle_count_proxy (traffic intensity)
    Roof_type_onehot (metal, bitumen, green, reflective) — aus Klassifikation
    Surface_albedo
    Distance_to_green_space
    Population_density, Median_income, %65plus (für equity)
    Wind_speed, humidity at measurement time

## Modell- und Validierungs-Details
block-spatial CV (z. B. K-fold mit räumlichen Clustern) verhindert Overfitting durch räumliche Nähe. Standardisiere Variablen, S enkrechte Trennung: LST vs. Lufttemperatur (beides unterschiedlich interpretierbar). Uncertainty quantification: prediction intervals (Quantile Regression Forests) oder ensemble

## Experimente / Fragestellungen 

- Wie stark erklärt Parkdichte lokale LST-Variationen, relativ zu Versiegelung?
- Welche Faktoren dominieren in dicht bebauten Gebieten vs. Wohngebieten?
- Gibt es systematische Unterschiede (Hitzeexposition) entlang sozioökonomischer Achsen?
- Modell, wie viel °C Einsparung eine bestimmte Maßnahme (z. B. 20% Permeabilitätserhöhung oder 30% Dachbegrünung) erzeugt.
- Robustheit: Klassifiziere Dächer aus Bilddaten vs. Katasterdaten — wie konsistent sind die Ergebnisse?


## Metriken & Karten
- Karten -> LST-Hotspots, Versiegelung, Parkdichte, Vulnerability overlay (Income / Age)
- Feature Importance (global + lokal), Partial Dependence, Szenario-Temperatur-Reduktion
- Validierung: spatial CV performance table, residual maps, Moran’s I